#    DESAFÍO 2
##   Modelo de Regresión
###  Grupo 4: Florencia Accardo, Juan Mutis, Joaquín Fernández, Rodrigo Arias, Ignacio Nasso 

<a id="section_toc"></a> 
## Indice

<a href="#section_intro">PARTE INICIAL. IMPORTACIÓN DE LIBRERÍAS. IMPORTACIÓN DE DATA FINAL DESAFÍO 1</a>

<a href="#section_variables">NUEVAS VARIABLES</a>

<a href="#section_dummies">GENERAMOS DUMMIES</a>

<a href="#section_estandar">ESTANDARIZAMOS VARIABLES NUMÉRICAS</a>

<a href="#section_data">PREPARAMOS DATA FRAME A INGRESAR EN LOS MODELOS</a>

<a href="#section_reg_sts">REGRESIÓN LINEAL CON STASTMODEL</a>

<a href="#section_reg_skl">REGRESIÓN LINEAL CON SKLEARN</a>

<a href="#section_vif">EVALUAMOS VIF</a>

<a href="#section_lasso_cv">REGULARIZAMOS CON LASSO</a>

<a href="#section_ridge_cv">REGULARIZAMOS CON RIDGE</a>

<a href="#section_dec_tree">PROBAMOS DECISION TREE</a>

<a href="#section_ensemble">PROBAMOS MODELOS DE ENSEMBLE</a>

<a href="#section_gauss">EVALUAMOS PRINCIPIOS GAUSS-MARKOV</a>


---

<a id="section_intro"></a>
#### Importamos los datos del Desafío 1 y las librerías a utilizar

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
import statsmodels.api as sm
from statsmodels.tools import eval_measures
from category_encoders import TargetEncoder
import re

In [ ]:
data = pd.read_csv('data_final')
data.head()

In [ ]:
data.info()

In [ ]:
#sacamos 2 nulos que nos habían quedado en descr clean
data.loc[data.description_Clean.isna(),"description_Clean"] = ""

<a id="section_variables"></a>
#### Genramos nuevas variables para obtener mejores predicciones

##### Nuevas variables dicotómicas con las amenites que encontramos en description

In [ ]:
amenities = ['balcon','toilette','suite', 'jardin', 'patio', 
             'dependencia', 'pileta', 'vigilancia', 'parrilla', 'ascensor', 'gimnasio', 'quincho', 'sauna', 'solarium', 'sala de juegos', 
             'laundry', 'sum', 'terraza', 'cochera', 'garage']

for i in amenities:
    data[i]= (data.description_Clean.str.contains(i)).astype(int)


##### Agregamos una columna que tenga el precio promedio por Property Type y state_name

In [ ]:
#Creamos una serie que tenga la media de precio por 'property_type' y 'state_name'
grouped_by_price = data.groupby(['property_type','state_name', 'Barrio_1', 'Barrio_2' ])['price_clean'].transform('median').round(0)

data['precio_promedio'] = grouped_by_price
data.columns

##### Regex para ver la cantidad de baños

In [ ]:
#extraccion de n de baños de la descripcion
banos_extract = data['description_Clean'].str.extract("([\d])(\s+)?(banos|bano|baño|baños)", re.IGNORECASE)

data["banos_Clean"] = banos_extract[0].str.replace(",","").replace(".","")

#eliminamos valores nulos
data["banos_Clean"] = data["banos_Clean"].apply(lambda x: x if x is not np.NaN else 0)

#convertimos a integer
data["banos_Clean"]=pd.to_numeric(data["banos_Clean"], errors="coerce", downcast="integer")

#reemplazamos donde banos son 0 y suponemos que tienen por lo menos 1
data.loc[data["banos_Clean"]==0,"banos_Clean"] = 1

#reemplazamos donde figura la palabra "banos" y solo figura 1, suponemos que tienen por lo menos 2
data.loc[(data["banos_Clean"]==1)&(data["description_Clean"].str.contains("banos")),"banos_Clean"] = 2

##### Generación de Target Encoder 

In [ ]:
#usamos target encoder para Barrio_1 / Barrio_2 / Barrio_3 / State_name

for col in ['state_name', 'Barrio_1', 'Barrio_2']:
  encoder = TargetEncoder()
  data[col+"_Target"] = encoder.fit_transform(data[col], data['surface_covered_in_m2_Clean'])
    
   #For the case of continuous target: features are replaced with a blend of the expected value of the target given particular 
#categorical value and the expected value of the target over all the training data.


In [ ]:
data.columns

In [ ]:
data.shape

##### Dropeamos las columnas que no vamos a meter dentro del modelo, ya sea por el valor que aportan o para no tener data leakage

In [ ]:
data.drop(["Unnamed: 0","description_Clean", 'price_clean', 'Barrio_3'],axis=1,inplace=True)

<a id="section_dummies"></a>
#### Generación de Dummies

In [ ]:
#Creamos una lista con las variables categoricas a transformar en dummies

categoricals = ['property_type', 'state_name', 'Barrio_1', 'Barrio_2']

#generamos un dataframe sólo con las categoricals

X_cat = data[categoricals]

#instanciamos el OneHotEncoder
#no usamos el drop = First porque en el paso siguiente dropeamos arbitrariamente distintas dummies, por lo que no necesitamos que 

enc = preprocessing.OneHotEncoder()

#fiteamos con X_cat (esto lo que hace es aprender todas las categorías posibles)

enc.fit(X_cat)

#listamos las categorías
enc.categories_


#generamos el dataframe de dummies

dummies = enc.transform(X_cat).toarray()
dummies_df = pd.DataFrame(dummies)

#Generamos los nombres de las columnas

col_names = [categoricals[i] + '_' + enc.categories_[i] for i in range(len(categoricals)) ]

#dropeamos los primeros nombres de las columnas, porque es lo que hicimos cuando instanciamos el OneHotEncoder

#col_names_drop_first = [sublist[i] for sublist in col_names for i in range(len(sublist)) if i != 0]

col_names_sin_drop = [sublist[i] for sublist in col_names for i in range(len(sublist))]


#renombramos las columnas en el dataframe de dummies

#dummies_df.columns = col_names_drop_first
dummies_df.columns = col_names_sin_drop


dummies_df.head()

In [ ]:
#Dropeamos arbitrariamente las dummies que tienen valores True por menos de 500 registros. Entendemos que si menos del 0.5%
# de los datos cuentan con esa característica, no nos proporciona info útil

columnas_dummies = dummies_df.columns
for i in columnas_dummies:
    if dummies_df[i].sum() < 500:
        dummies_df.drop(i, axis = 1, inplace = True)



<a id="section_estandar"></a>
#### Estandarizamos las variables numéricas

In [ ]:

#listamos todas las variables numericas
numericas = ['surface_total_in_m2_Clean', 'surface_covered_in_m2_Clean', 'precio_promedio','rooms_Clean', 'banos_Clean', 'state_name_Target','Barrio_1_Target', 'Barrio_2_Target']

#instanciamos el scaler

#scaler_std = preprocessing.StandardScaler()
min_max_scaler = preprocessing.MinMaxScaler()
#generamos el dataframe a estandarizar

X_num = data[numericas]

#fiteamos con el scaler

min_max_scaler.fit(X_num)

#transoformamos y generamos el data frame con los valores estandarizados

X_scal = min_max_scaler.transform(X_num)
X_scal_df = pd.DataFrame(X_scal)

#renombramos las columnas del DF con el agregado de 'std' para identificarlas
X_scal_df.columns = [i + '_std' for i in numericas]
X_scal_df

<a id="section_data"></a>
#### Preparamos el dataframe que va a entrar al modelo

##### Unimos data, dummies y normalizadas

In [ ]:
#unimos data a dummies_df, X_scal_df y dropeamos lo que no necesitamos mas en data 

data = pd.concat([data, X_scal_df, dummies_df], axis = 1)

data.drop(['property_type', 'surface_total_in_m2_Clean', 'surface_covered_in_m2_Clean', 
          'precio_promedio', 'state_name', 'Barrio_1', 'Barrio_2', 'rooms_Clean', 'banos_Clean'], axis=1, inplace = True)

data.head()

##### Train Test Split

In [ ]:
col_feature = ['balcon', 'toilette', 'suite', 'jardin',
       'patio', 'dependencia', 'pileta', 'vigilancia', 'parrilla', 'ascensor',
       'gimnasio', 'quincho', 'sauna', 'solarium', 'sala de juegos', 'laundry',
       'sum', 'terraza', 'cochera', 'garage', 'surface_total_in_m2_Clean_std',
       'surface_covered_in_m2_Clean_std', 'precio_promedio_std',
       'rooms_Clean_std', 'banos_Clean_std', 'property_type_PH',
       'property_type_apartment', 'property_type_house', 'property_type_store',
       'state_name_Bs.As. Costa Atlántica',
       'state_name_Bs.As. G.B.A. Zona Norte',
       'state_name_Bs.As. G.B.A. Zona Oeste',
       'state_name_Bs.As. G.B.A. Zona Sur', 'state_name_Bs.As. Interior',
       'state_name_Capital Federal', 'state_name_Córdoba',
       'state_name_Mendoza', 'state_name_Neuquén', 'state_name_Río Negro',
       'state_name_Santa Fe', 'state_name_Tucumán', 'Barrio_1_Almagro',
       'Barrio_1_Almirante Brown', 'Barrio_1_Avellaneda', 'Barrio_1_Balvanera',
       'Barrio_1_Barrio Norte', 'Barrio_1_Belgrano', 'Barrio_1_Caballito',
       'Barrio_1_Córdoba', 'Barrio_1_Escobar', 'Barrio_1_Flores',
       'Barrio_1_General San Martín', 'Barrio_1_Ituzaingó',
       'Barrio_1_La Matanza', 'Barrio_1_La Plata', 'Barrio_1_Lanús',
       'Barrio_1_Lomas de Zamora', 'Barrio_1_Mar del Plata', 'Barrio_1_Morón',
       'Barrio_1_Nuñez', 'Barrio_1_Palermo', 'Barrio_1_Pilar',
       'Barrio_1_Pinamar', 'Barrio_1_Quilmes', 'Barrio_1_Recoleta',
       'Barrio_1_Rosario', 'Barrio_1_San Fernando', 'Barrio_1_San Isidro',
       'Barrio_1_San Miguel', 'Barrio_1_San Telmo', 'Barrio_1_Santa Fe',
       'Barrio_1_Sin definir', 'Barrio_1_Tigre', 'Barrio_1_Tres de Febrero',
       'Barrio_1_Vicente López', 'Barrio_1_Villa Crespo',
       'Barrio_1_Villa Urquiza', 'Barrio_2_Adrogué', 'Barrio_2_Banfield',
       'Barrio_2_Castelar', 'Barrio_2_La Plata', 'Barrio_2_Lanús',
       'Barrio_2_Lomas de Zamora', 'Barrio_2_Martínez', 'Barrio_2_Morón',
       'Barrio_2_Nordelta', 'Barrio_2_Olivos', 'Barrio_2_Ramos Mejía',
       'Barrio_2_Sin definir', 'Barrio_2_Temperley', 'Barrio_2_Tigre',
       'Barrio_2_Villa Ballester', 'state_name_Target','Barrio_1_Target', 'Barrio_2_Target']

col_target = "price_usd_per_m2_Clean"

X = data[col_feature]

y = data[col_target]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1234)

<a id="section_reg_sts"></a>
#### LinearRegression con StatsModel

In [ ]:
#sumamos la constante a las features de train y test 

X_train_sm = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)

#instanciamos y fiteamos el modelo en el mismo paso

linereg_sm = sm.OLS(y_train, X_train_sm).fit()

#Vemos el summary

linereg_sm.summary()

In [ ]:
#vemos las predecimos y evaluamos en train 

sm_prediction_train = linereg_sm.predict(X_train_sm)
print(eval_measures.rmse(y_train, sm_prediction_train))


sm_prediction_test = linereg_sm.predict(X_test_sm)
print(eval_measures.rmse(y_test, sm_prediction_test))



<a id="section_reg_skl"></a>
#### LinearRegression Multiple con Sklearn

In [ ]:
def regLinMul_function(dataset, col_feature, col_target):
 
    reg = linear_model.LinearRegression()

    # Entrenamos el modelo 

    reg.fit(X_train, y_train)

    #realizamos predicciones con el set de testeo
    y_pred= reg.predict(X_test)
    
    
    #vemos los Betas

    # print (reg.intercept_)
    # print (reg.coef_)

    # print(list(zip(X.columns, reg.coef_)))

    #evaluamos el r2 en Train 

    print ('R2 en Train:', reg.score(X_train, y_train))
    
    # evaluamos el r2 en Train ajustado por la cantidad de datos y la cantidad de features
    r2_train = reg.score(X_train, y_train)
    n = data.shape[0]
    p = len(col_feature)
    #evaluamos el r2 en Test
    print ('R2 en Test:', reg.score(X_test, y_test))
    print ('MAE:', metrics.mean_absolute_error(y_test,y_pred))
    print ('MSE:', metrics.mean_squared_error(y_test,y_pred))
regLin = regLinMul_function(data,X,y)

<a id="section_vif"></a>
#### Evaluamos el VIF

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = data[col_feature]
y = data[col_target]


series_vif =  pd.Series([variance_inflation_factor(X.values,i) for i in range(X.shape[1])], index=X.columns)

series_vif.sort_values(ascending=False).head(20)

<a id="section_lasso_cv"></a>
#### Lasso con Cross Validation

In [ ]:
#instanciamos el modelo y le metemos varios alphas para que pruebe 

lassoCV = linear_model.LassoCV(alphas = np.arange(0.001, 1, 0.005), cv = 3, normalize = False)

#fiteamos con train

lasso_cv_model = lassoCV.fit(X_train, y_train)

#el mejor alpha que encontró con Cross Validation 

print('Best alpha', lasso_cv_model.alpha_)
best_alpha_lasso = lasso_cv_model.alpha_

#Vemos Beta 0 y los coeficientes 

#print(lasso_cv_model.intercept_)
#print(lasso_cv_model.coef_)

#realizamos predicciones con el set de testeo
y_pred= lassoCV.predict(X_test)

#Observamos las metricas
print("R2 Train",lasso_cv_model.score(X_train, y_train))
print('R2 Test:', lasso_cv_model.score(X_test, y_test))
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test,y_pred))

<a id="section_ridge_cv"></a>
#### Ridge con Cross Validation

In [ ]:
#instanciamos el modelo y le metemos varios alphas para que pruebe 

ridgeCV = linear_model.RidgeCV(alphas = np.arange(1, 5, 0.01), normalize = False)

#fiteamos con train

ridge_cv_model = ridgeCV.fit(X_train, y_train)

#el mejor alpha que encontró con Cross Validation 

print('Best alpha', ridge_cv_model.alpha_)
#best_alpha_ridge = ridge_cv_model.alpha_

#Vemos Beta 0 y los coeficientes 

#ridge_cv_model.intercept_
#ridge_cv_model.coef_

#realizamos predicciones con el set de testeo
y_pred= ridge_cv_model.predict(X_test)

#Observamos las metricas
print("R2 Train",ridge_cv_model.score(X_train, y_train))
print('R2 Test:', ridge_cv_model.score(X_test, y_test))
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test,y_pred))

<a id="section_dec_tree"></a>
#### Probamos con DecisionTreeRegressor

In [ ]:
from sklearn import tree
reg_tree = tree.DecisionTreeRegressor(random_state=1234)
reg_tree.fit(X_train,y_train)
#Observamos features mas importantes
sorted(list(zip(X.columns,reg_tree.feature_importances_)),key=lambda x: x[1], reverse=True)[:5]
#realizamos predicciones con el set de testeo
y_pred= reg_tree.predict(X_test)

#Observamos las metricas
print("R2 Train",reg_tree.score(X_train, y_train))
print('R2 Test:', reg_tree.score(X_test, y_test))
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test,y_pred))

<a id="section_ensemble"></a>
### Ensemble

##### HistGradientBoostingRegressor

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
HGBreg=HistGradientBoostingRegressor(random_state=1234)
HGBreg.fit(X_train,y_train)

#realizamos predicciones con el set de testeo
y_pred= HGBreg.predict(X_test)

#Observamos las metricas
print("R2 Train",HGBreg.score(X_train, y_train))
print('R2 Test:', HGBreg.score(X_test, y_test))
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test,y_pred))

##### RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RFreg=RandomForestRegressor(random_state=1234,n_estimators=100,max_depth=10)
RFreg.fit(X_train,y_train) 

#realizamos predicciones con el set de testeo
y_pred= RFreg.predict(X_test)

#Observamos las metricas
print("R2 Train",RFreg.score(X_train, y_train))
print('R2 Test:', RFreg.score(X_test, y_test))
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test,y_pred))

<a id="section_gauss"></a>

#### Supuestos de GAUSS MARKOV

##### Linealidad del modelo

In [ ]:
%matplotlib inline
#%config InlineBackend.figure_format ='retina'
import seaborn as sns 
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
sns.set_style('darkgrid')
sns.mpl.rcParams['figure.figsize'] = (15.0, 9.0)

def linearity_test(model, y):
    '''
    funcion para visualizar e identificar supuestos de linealidad sobre la regression lineal
    Args:
    * model - fitted OLS model from statsmodels
    * y - observed values
    '''
    fitted_vals = model.predict()
    resids = model.resid

    fig, ax = plt.subplots(1,2)
    
    sns.regplot(x=fitted_vals, y=y, lowess=True, ax=ax[0], line_kws={'color': 'red'})
    ax[0].set_title('Observados vs. Valores Predichos', fontsize=16)
    ax[0].set(xlabel='Predichos', ylabel='Observados')

    sns.regplot(x=fitted_vals, y=resids, lowess=True, ax=ax[1], line_kws={'color': 'red'})
    ax[1].set_title('Residos vs. Valores Predichos', fontsize=16)
    ax[1].set(xlabel='Predichos', ylabel='Residuos')
    
linearity_test(linereg_sm, y_train)    

Observar que el patrón inclinado en la grafica de los residuos puede deberse a que el modelo contiene muchas características

##### Esperanza de los residuos igual a cero.

In [ ]:
linereg_sm.resid.mean()

##### Multicolinealidad

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = [variance_inflation_factor(X_train_sm.values, i) for i in range(X_train_sm.shape[1])]
pd.DataFrame({'vif': vif[1:]}, index=X.columns).T

##### Homocedasticidad 

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'
import seaborn as sns 
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
sns.set_style('darkgrid')
sns.mpl.rcParams['figure.figsize'] = (15.0, 9.0)

def homoscedasticity_test(model):
    '''
    Function for testing the homoscedasticity of residuals in a linear regression model.
    It plots residuals and standardized residuals vs. fitted values and runs Breusch-Pagan and Goldfeld-Quandt tests.
    
    Args:
    * model - fitted OLS model from statsmodels
    '''
    import numpy as np
    fitted_vals = model.predict()
    resids = model.resid
    resids_standardized = model.get_influence().resid_studentized_internal

    fig, ax = plt.subplots(1,2)

    sns.regplot(x=fitted_vals, y=resids, lowess=True, ax=ax[0], line_kws={'color': 'red'})
    ax[0].set_title('Residuals vs Fitted', fontsize=16)
    ax[0].set(xlabel='Fitted Values', ylabel='Residuals')

    sns.regplot(x=fitted_vals, y=np.sqrt(np.abs(resids_standardized)), lowess=True, ax=ax[1], line_kws={'color': 'red'})
    ax[1].set_title('Scale-Location', fontsize=16)
    ax[1].set(xlabel='Fitted Values', ylabel='sqrt(abs(Residuals))')

    bp_test = pd.DataFrame(sms.het_breuschpagan(resids, model.model.exog), 
                           columns=['value'],
                           index=['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value'])

    gq_test = pd.DataFrame(sms.het_goldfeldquandt(resids, model.model.exog)[:-1],
                           columns=['value'],
                           index=['F statistic', 'p-value'])

    print('\n Breusch-Pagan test ----')
    print(bp_test)
    print('\n Goldfeld-Quandt test ----')
    print(gq_test)
    print('\n Residuals plots ----')

homoscedasticity_test(linereg_sm)

##### Autocorrelación

In [ ]:
import statsmodels.tsa.api as smt

acf = smt.graphics.plot_acf(linereg_sm.resid, lags=40, alpha=0.05)

In [ ]:
from scipy import stats

def normality_of_residuals_test(model):
    '''
    Function for drawing the normal QQ-plot of the residuals and running 4 statistical tests to 
    investigate the normality of residuals.
    
    Arg:
    * model - fitted OLS models from statsmodels
    '''
    sm.ProbPlot(model.resid).qqplot(line='s');
    plt.title('Q-Q plot');

    jb = stats.jarque_bera(model.resid)
    sw = stats.shapiro(model.resid)
    ad = stats.anderson(model.resid, dist='norm')
    ks = stats.kstest(model.resid, 'norm')
    
    print(f'Jarque-Bera test ---- statistic: {jb[0]:.4f}, p-value: {jb[1]}')
    print(f'Shapiro-Wilk test ---- statistic: {sw[0]:.4f}, p-value: {sw[1]:.4f}')
    print(f'Kolmogorov-Smirnov test ---- statistic: {ks.statistic:.4f}, p-value: {ks.pvalue:.4f}')
    print(f'Anderson-Darling test ---- statistic: {ad.statistic:.4f}, 5% critical value: {ad.critical_values[2]:.4f}')
    print('If the returned AD statistic is larger than the critical value, then for the 5% significance level, the null hypothesis that the data come from the Normal distribution should be rejected. ')
    
normality_of_residuals_test(linereg_sm)